In [1]:
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import copy, time
import random
import pickle
import scipy

import timeit

import mlrfit as mf
import mfmodel as mfm
import numba as nb

from scipy.sparse import coo_matrix
from scipy.linalg import block_diag, pinvh

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(1001)
random.seed(1001)

# True model is MLR factor model with SNR = 4 

Let $\Sigma = FF^T + D$ be MLR. We generate samples using 
$$
y = Fz + e, \qquad z \sim N(0, I), \qquad N(0, D).
$$

In [3]:
mtype = "small_mlr_hier"
n = 100000
signal_to_noise = 4



L = 5

# ranks = np.array([5, 4, 3, 2, 1])
ranks = np.array([30, 20, 10, 5, 1])
rank = ranks.sum()

nsamples = rank * 4
nsamples

264

In [4]:
pi_rows = np.random.permutation(n)
hpart = {'rows':{'pi':pi_rows, 'lk':[]}, 'cols':{'pi':pi_rows, 'lk':[]}} 
for ngroups in [2, 5, 9, 17, n+1]:
       hpart['rows']['lk'] += [ np.linspace(0, n, ngroups, endpoint=True, dtype=int)]
hpart['rows']['lk'][1] = np.delete(hpart['rows']['lk'][1], -2)
hpart['rows']['lk'][2] = np.delete(hpart['rows']['lk'][2], -4)
hpart['cols']['lk'] = hpart['rows']['lk']
part_sizes = mfm.print_hpart_numgroups(hpart)
mfm.valid_hpart(hpart)

level=0, num_groups=1, mean_size=100000.0
level=1, num_groups=3, mean_size=33333.3
level=2, num_groups=7, mean_size=14285.7
level=3, num_groups=16, mean_size=6250.0
level=4, num_groups=100000, mean_size=1.0


In [5]:
F_hpart = {"pi": hpart['rows']["pi"], "lk": hpart['rows']["lk"][:-1]}
true_mfm = mfm.MFModel()
true_mfm = mfm.generate_mfmodel(true_mfm, n, F_hpart, ranks, signal_to_noise, debug=False)
F_hpart["pi_inv"] = true_mfm.pi_inv

signal_var=81.15506713197232, noise_var=16.242432778950498
SNR=3.9964847160793444, signal_to_noise=4


In [6]:
n, true_mfm.num_factors(), L, ranks.sum()

(100000, 240, 5, 66)

In [7]:
v = np.random.randn(n, 1)


hat_x = true_mfm.solve(v, eps=1e-12, max_iter=20, printing=False)
reldiff = np.linalg.norm(true_mfm.matvec(hat_x) - v) / np.linalg.norm(v)
print(f"solve {reldiff=}")

solve reldiff=9.744615586746802e-13


In [8]:
%timeit true_mfm.solve(v, eps=1e-12, max_iter=20, printing=False)

2.56 s ± 103 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
true_mfm.D.min(), true_mfm.D.max()

(0.0006785230567792892, 32.456283388481424)

# EM

In [10]:
C = mfm.sample_data(nsamples, true_mfm)

Z = (C - C.mean(axis=1, keepdims=True))[F_hpart["pi"], :]
# unpermuted_A = (Z @ Z.T / (Z.shape[1]-1))[true_mfm.pi_inv, :][:, true_mfm.pi_inv]

# permute to put clusters on diagonal
Y = Z.T
N = Y.shape[0]

permuted_F_hpart = {"pi_inv":np.arange(n), "pi":np.arange(n), "lk":F_hpart["lk"]}
row_selectors, si_groups, F_hpart, groups_all = mfm.row_col_selections(hpart, return_groups=True)


mfm_Sigma = mfm.MFModel(hpart=F_hpart, ranks=ranks)
mfm_Sigma.init_FD(ranks, F_hpart, init_type="Y", Y=Y)
F0, D0 = mfm_Sigma.F, mfm_Sigma.D

num_levels=5, num_sparsities=16
(16, 4) [ 0  2  6 15]


In [12]:
rank, mfm_Sigma.num_factors()

(66, 240)

In [37]:
sigmas+1

array([1.05368833, 1.05523631, 1.05674229, 1.05750556, 1.05773613,
       1.06071419, 1.06110294, 1.06155948, 1.06194614, 1.06248022,
       1.06332585, 1.06386022, 1.06448366, 1.06477398, 1.06535907,
       1.06644549, 1.0667757 , 1.06691082, 1.0677316 , 1.06928968,
       1.07014855, 1.07072819, 1.07101891, 1.07129875, 1.07236872,
       1.07272367, 1.07321637, 1.07352292, 1.07394081, 1.07414381,
       1.07505886, 1.07527753, 1.07610143, 1.07660426, 1.07709938,
       1.07746462, 1.07867053, 1.07965749, 1.08060884, 1.08068068,
       1.08099143, 1.08230649, 1.08271157, 1.08343153, 1.0847472 ,
       1.08538202, 1.08574303, 1.08590256, 1.08626487, 1.08648889,
       1.08762985, 1.08890723, 1.08935738, 1.08994899, 1.09035206,
       1.0910507 , 1.09126126, 1.09170935, 1.09188081, 1.09216296,
       1.09276781, 1.09332395, 1.09360047, 1.09398573, 1.09468198,
       1.09528424, 1.09563682, 1.09609575, 1.09650034, 1.09662218,
       1.09690574, 1.09712681, 1.0973437 , 1.09805473, 1.09934

In [32]:
rescaled_sparse_F.shape

(100000, 240)

array([0.0516795])

In [ ]:
rescaled_sparse_F

In [35]:
sigmas

array([0.05368833, 0.05523631, 0.05674229, 0.05750556, 0.05773613,
       0.06071419, 0.06110294, 0.06155948, 0.06194614, 0.06248022,
       0.06332585, 0.06386022, 0.06448366, 0.06477398, 0.06535907,
       0.06644549, 0.0667757 , 0.06691082, 0.0677316 , 0.06928968,
       0.07014855, 0.07072819, 0.07101891, 0.07129875, 0.07236872,
       0.07272367, 0.07321637, 0.07352292, 0.07394081, 0.07414381,
       0.07505886, 0.07527753, 0.07610143, 0.07660426, 0.07709938,
       0.07746462, 0.07867053, 0.07965749, 0.08060884, 0.08068068,
       0.08099143, 0.08230649, 0.08271157, 0.08343153, 0.0847472 ,
       0.08538202, 0.08574303, 0.08590256, 0.08626487, 0.08648889,
       0.08762985, 0.08890723, 0.08935738, 0.08994899, 0.09035206,
       0.0910507 , 0.09126126, 0.09170935, 0.09188081, 0.09216296,
       0.09276781, 0.09332395, 0.09360047, 0.09398573, 0.09468198,
       0.09528424, 0.09563682, 0.09609575, 0.09650034, 0.09662218,
       0.09690574, 0.09712681, 0.0973437 , 0.09805473, 0.09934

In [11]:
loglikelihoods = [-np.inf]
N = Y.shape[0]
eps = 1e-12
for t in range(1):
    F1 = mfm.fast_EM_get_F(F0, D0, Y, ranks, permuted_F_hpart, row_selectors, si_groups)
    D1 = mfm.fast_EM_get_D(F0, D0, F1, Y, ranks, permuted_F_hpart, row_selectors, si_groups)
    F0, D0 = F1, D1
    assert D1.min() >= -1e-8 #and loglikelihoods[-2] - 1e-8 <= loglikelihoods[-1]
    if t % 1 == 0:
        print(f"{t=},  {D1.min()=}, {D1.max()=}")


print("PASSED")

t=0,  D1.min()=2.804383284514381, D1.max()=66.71522417869596
t=1,  D1.min()=0.3804383003692127, D1.max()=44.27694622515326
t=2,  D1.min()=0.16481087561646057, D1.max()=41.762775694112534
t=3,  D1.min()=0.06653720385368918, D1.max()=40.55538386773556
t=4,  D1.min()=0.044201850472883525, D1.max()=41.11083985415371


KeyboardInterrupt: 